In [1]:
# baseball-regression.ipynb
# Alexis Perumal, Venkat Pinnika, Young You, 1/8/2020
#
# Objective:
# . Build a predictor for the winner of baseball games (visitor or home)
#   with > 50% prediction accuracy, with statistical accuracy.
# . Analysis will be done by season, looking at all the regular season
#   games of a given year with each days' game predicted based on info
#   up to, but not including that day.
#
# Strategy:
# . Build a dataframe of games (rows) and factors to build a regression
#   model for visiting net points, and by extension, predict the winner
#   of each game (visitor or home).
# . Explore candidate factors with regression analysis (T-stat, F-stat,
#   p-value), build and optimize a simple LSR model using a training set
#   through the 2017 Season.
# . Plot the results.
# . Ultimately, apply the model to the 2018, then 2019 seasons.
#
# 1/9/20 - Updating to add pitcher-based prediction
# 1/10/20 - Coding different values as columns and then running a linear regression on them.
# 1/11/20 - Cleaned up a defect where it was using the wrong model outputs to calc.
#         - replaced manual prediction with call to .predict()

In [2]:
# Modules
import os
import csv
import pprint
import pandas as pd
import glob
import pprint
import datetime
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
%matplotlib inline
!pwd

/Users/aperumal/OneDrive/Personal/UCSD_Data_Science_Bootcamp/Homework/2019-12-18_Proj1_Baseball-Predictor/baseball-predictor/alexis-regression


In [3]:
# Support functions
def date_str(date: int):
    s = str(date)
    return f"{s[0:4]}-{s[4:6]}-{s[6:]}"


# Passed the gamedays series (YYYYMMDD) and season (year) return with the first
# and last game dates.
def date_range(gamedays, season):
    # Converts 'YYYY' string to an integer start date, YYYY0101 and end date YYYY1231
    def season_to_date(season): 
#         return (int(season)*10000 + 101, int(season)*10000+1231)
        # Hack to shorten the season for dev purposes (faster analysis)
        return (int(season)*10000 + 101, int(season)*10000+430)
    
    first_of_year, last_of_year = season_to_date(season)
    season_gamedays = gamedays.loc[((gamedays >= first_of_year) &
                                    (gamedays <= last_of_year))]
    return (season_gamedays.iloc[0], season_gamedays.iloc[-1])
    

# returns new game date offset by n. Passed in the gamedays series.
def gamedays_offset(gamedays, base_date, n): 
    if base_date not in gamedays.values:
        raise ValueError(f"{base_date}, not in the the gamedays series.")
    base_date_index = gamedays[gamedays==base_date].index[0]
    if ((n + base_date_index) < 0) or n + base_date_index >= len(gamedays):
        raise ValueError(f"Attempting to calculate a game date outside the dataset.")
        return(0)  # Out of range
    else:
        new_index = base_date_index + n
        return gamedays.iloc[new_index]
    
def derive_metrics(results_df):
    num_games = len(results_df)
    num_correct = results_df['Prediction Correct?'].values.sum()
    percent_correct = num_correct/num_games*100.
    return (num_games, num_correct, percent_correct)

In [4]:
def read_source_data():
    def reader(f):
        df = pd.read_csv(f, index_col=False, header=None)  
        df.columns = [("Col_"+str(i)) for i in range(1,df.shape[1]+1)]       
        return df

    files = glob.glob("../datasets/Final_Data_Files/GL*.csv")
    files.sort()
    df = pd.concat([reader(f) for f in files])
    old_df_len = len(df)
    
    # Insert column headers
    df = df.rename(columns={'Col_1':'Date',
                            'Col_4':'Visiting Team',
                            'Col_5':'Visiting League',
                            'Col_7':'Home Team',
                            'Col_8':'Home League',
                            'Col_10':'Visiting Score',
                            'Col_11':'Home Score',
                            'Col_102':'V Start Pitcher ID',
                            'Col_104':'H Start Pitcher ID'
                           }).reset_index()
    df = df[['Date', 'Visiting Team', 'Visiting League', 'Home Team', 'Home League',
            'Visiting Score','Home Score', 'V Start Pitcher ID','H Start Pitcher ID']]
    
    df = df.replace('FLO','MIA') # After the 2011 season, the Florida Marlins
                                 # rebranded themselves the Miami Marlins. This
                                 # search and replace makes the two the same.
    
    # Drop all rows with missing information
    # print(df.head())
    df = df.dropna(how='any')
    if len(df) < old_df_len:
        print(f"Dropped {old_df_len-len(df)} rows due to missing data.")
    
    # Create new columns we'll need. 
#     df['Key'] = df['Date'].astype(str) + df['Visiting Team']
    df['Home Winner'] = df['Home Score'] > df['Visiting Score']
    df['V NetRuns'] = df['Visiting Score'] - df['Home Score']
    df['H NetRuns'] = - df['V NetRuns']
    
    #     print(df.shape)
    print(f"Dataset loaded with {df.shape[0]} games, ", end='')
    print(f"{df.shape[1]} columns, {date_str(df.iloc[0, 0])} - ", end='')
    print(f"{date_str(df.iloc[-1,0])}")
    return df

# read_source_data()

In [5]:
# Add columns to the dataset that will be used for regression analysis.

# Get the source dataset.
g = read_source_data()
g.index.set_names('games_idx', inplace=True)  # Name the row index column so we can sort by it

# Calculate baseline stats on home wins.
num_games = len(g)
home_wins = g['Home Winner'].sum()
home_wins_percent = home_wins / num_games * 100.
print(f'Baseline results: {num_games} games, {home_wins} home wins = {round(home_wins_percent, 2)}%')

# Calculate rolling average of runs allowed by starting pitcher excluding the current row (shift=prior_exclude+1)
lookback_n = 10
g = g.sort_values(by = ['Visiting Team','games_idx'])
g['V Runs Avg'] = g.groupby('Visiting Team')['Visiting Score'].transform(lambda x: x.shift(periods=1).rolling(window=lookback_n).mean())
g = g.sort_values(by = ['Home Team','games_idx'])
g['H Runs Avg'] = g.groupby('Home Team')['Home Score'].transform(lambda x: x.shift(periods=1).rolling(window=lookback_n).mean())
g = g.sort_index()

# Calculate rolling average of runs allowed by starting pitcher excluding the current row (shift=prior_exclude+1)
lookback_n = 10
g = g.sort_values(by = ['V Start Pitcher ID', 'games_idx'])
g['V Runs Allowed'] = g.groupby('V Start Pitcher ID')['Home Score'].transform(lambda x: x.shift(periods=1).rolling(window=lookback_n).mean())
g = g.sort_values(by = ['H Start Pitcher ID', 'games_idx'])
g['H Runs Allowed'] = g.groupby('H Start Pitcher ID')['Visiting Score'].transform(lambda x: x.shift(periods=1).rolling(window=lookback_n).mean())
g = g.sort_index()

g

Dataset loaded with 19437 games, 12 columns, 2010-04-04 - 2017-10-01
Baseline results: 19437 games, 10438 home wins = 53.7%


,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,V Start Pitcher ID,H Start Pitcher ID,Home Winner,V NetRuns,H NetRuns,V Runs Avg,H Runs Avg,V Runs Allowed,H Runs Allowed
games_idx,,,,,,,,,,,,,,,,
0,20100404,NYA,AL,BOS,AL,7,9,sabac001,beckj002,True,-2,2,NaN,NaN,NaN,NaN
1,20100405,MIN,AL,ANA,AL,3,6,bakes002,weavj003,True,-3,3,NaN,NaN,NaN,NaN
2,20100405,CLE,AL,CHA,AL,0,6,westj001,buehm001,True,-6,6,NaN,NaN,NaN,NaN
3,20100405,DET,AL,KCA,AL,8,4,verlj001,greiz001,False,4,-4,NaN,NaN,NaN,NaN
4,20100405,SEA,AL,OAK,AL,5,3,hernf002,sheeb001,False,2,-2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19432,20171001,ARI,NL,KCA,AL,14,2,ray-r002,vargj001,False,12,-12,3.4,4.2,2.3,4.4
19433,20171001,DET,AL,MIN,AL,1,5,sanca004,colob001,True,-4,4,2.4,6.9,6.2,6.9
19434,20171001,TOR,AL,NYA,AL,2,1,andeb004,montj004,False,1,-1,4.4,5.7,4.3,4.4


# Multiple Regression Analysis

In [14]:
# Perform regression analysis.
num_before_drop = len(g)
g = g.dropna(how='any')
num_after_drop = len(g)
print(f"Dropped {num_before_drop-num_after_drop} rows due to missing data.")

# The data table is built, now let's do the regression or raw points for Visitors
v_y = g['Visiting Score']
v_x = g[['V Runs Avg', 'H Runs Allowed']]
v_x = sm.add_constant(v_x)
v_est = sm.OLS(v_y,v_x).fit()
# g['V Score Predicted'] = v_est.params[0] + v_est.params[1]*g['V Runs Avg'] + v_est.params[2]*g['H Runs Allowed']
# g['V Score Predicted'] = v_est.predict()
print(v_est.params)
v_est.summary()


Dropped 0 rows due to missing data.
const             2.613831
V Runs Avg        0.099613
H Runs Allowed    0.284980
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Visiting Score   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     66.61
Date:                Sat, 11 Jan 2020   Prob (F-statistic):           1.75e-29
Time:                        08:18:17   Log-Likelihood:                -28323.
No. Observations:               11160   AIC:                         5.665e+04
Df Residuals:                   11157   BIC:                         5.667e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.6138      0.159     16.435      0.000       2.302       2.926
V Runs Avg         0.0996      0.027      3.628      0.000       0.046       0.153
H Runs Allowed     0.2850      0.026     10.833      0.000       0.233       0.337
==============================================================================
Omnibus:                     1700.657   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2816.425
Skew:                           1.026   Prob(JB):                         0.00
Kurtosis:                       4.359   Cond. No.                         33.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# The data table is built, now let's do the regression or raw points for HOME
h_y = g['Home Score']
h_x = g[['H Runs Avg', 'V Runs Allowed']]
h_x = sm.add_constant(h_x)
h_est = sm.OLS(h_y,h_x).fit()
# g['H Score Predicted'] = h_est.params[0] + h_est.params[1]*g['H Runs Avg'] + h_est.params[2]*g['V Runs Allowed']
# g['H Score Predicted'] = h_est.predict()
print(h_est.params)
h_est.summary()

const             2.161363
H Runs Avg        0.299314
V Runs Allowed    0.203596
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Home Score   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     100.6
Date:                Sat, 11 Jan 2020   Prob (F-statistic):           5.08e-44
Time:                        08:18:18   Log-Likelihood:                -27931.
No. Observations:               11160   AIC:                         5.587e+04
Df Residuals:                   11157   BIC:                         5.589e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.1614      0.161     13.465      0.000       1.847       2.476
H Runs Avg         0.2993      0.025     11.868      0.000       0.250       0.349
V Runs Allowed     0.2036      0.027      7.411      0.000       0.150       0.257
==============================================================================
Omnibus:                     1629.134   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2748.054
Skew:                           0.976   Prob(JB):                         0.00
Kurtosis:                       4.449   Cond. No.                         36.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Generate predictions
g2 = g.copy(deep=True)
g2['V Score Predicted'] = v_est.predict()
g2['H Score Predicted'] = h_est.predict()
g2.head()

,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,V Start Pitcher ID,H Start Pitcher ID,Home Winner,V NetRuns,H NetRuns,V Runs Avg,H Runs Avg,V Runs Allowed,H Runs Allowed,V Score Predicted,H Score Predicted
games_idx,,,,,,,,,,,,,,,,,,
1306,20100710,ATL,NL,NYN,NL,4,0,hudst001,pelfm001,False,4,-4,3.7,4.7,3.1,2.6,3.723347,4.199287
1393,20100720,CLE,AL,MIN,AL,4,3,mastj001,slowk001,False,1,-1,4.8,4.9,6.4,4.3,4.317387,4.931015
1404,20100720,PHI,NL,SLN,NL,1,7,moyej001,carpc002,True,-6,6,4.5,5.0,5.4,3.8,4.145013,4.757351
1425,20100722,SFN,NL,ARI,NL,3,0,cainm001,loper001,False,3,-3,6.5,4.8,3.6,5.5,4.828705,4.331016
1431,20100722,PHI,NL,SLN,NL,2,0,hamec001,waina001,False,2,-2,4.2,5.9,4.1,1.6,3.488173,4.762059


In [17]:
g2['Predict Home Wins?'] = g2['H Score Predicted'] >= g2['V Score Predicted']
g2['Prediction Correct'] = g2['Predict Home Wins?'] == g2['Home Winner']

# Calculate baseline stats on home wins.
num_games_now = len(g2)
num_correct = g2['Prediction Correct'].sum()
num_correct_percent = num_correct / num_games_now * 100.
print(f'Baseline results: {num_games} games, {home_wins} home wins = {round(home_wins_percent, 2)}%')
print(f'Baseline results: {num_games_now} games, {num_correct} predicted correctly = {round(num_correct_percent, 2)}%')


Baseline results: 19437 games, 10438 home wins = 53.7%
Baseline results: 11160 games, 5903 predicted correctly = 52.89%


In [18]:
print(f"Across the dataset: Visiting avg = {g['Visiting Score'].mean()}, Home avg = {g['Home Score'].mean()}")

Across the dataset: Visiting avg = 4.2027777777777775, Home avg = 4.335125448028674


# Individual Regression Analysis
Ignore everything from here down. It is older code no longer needed, but kept for our reference.

In [ ]:
# Perform regression analysis.
num_before_drop = len(g)
g = g.dropna(how='any')
num_after_drop = len(g)
print(f"Dropped {num_after_drop-num_before_drop} rows due to missing data.")

# The data table is built, now let's do the regression or raw points for Visitors
x = g['V Runs Avg']
y = g['Visiting Score']
slope, y_int, r, p, se = stats.linregress(x, y)

print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")


In [ ]:
# The data table is built, now let's do the regression or raw points for Home
x = g['H Runs Avg']
y = g['Home Score']
slope, y_int, r, p, se = stats.linregress(x, y)

print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")


In [ ]:
# Do a regression on runs given up
x = g['V Runs Allowed']
y = g['Home Score']
slope, y_int, r, p, se = stats.linregress(x, y)

print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")


In [ ]:
# Do a regression on runs given up
x = g['H Runs Allowed']
y = g['Visiting Score']
slope, y_int, r, p, se = stats.linregress(x, y)

print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")


In [ ]:
# Do a regression on runs given up, doesn't make sense.
x = g['V Runs Allowed']
y = g['Visiting Score']
slope, y_int, r, p, se = stats.linregress(x, y)

print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")


In [ ]:
# Do multiple regression


# Legacy code below for point predictors. Recommend don't run anything below this point.


In [ ]:
# Given a games df with predictions and prediction resuls by game already captured,
# generate an outcomes dataframe with stats by season.
def append_outcomes(outcomes_df, g, ra_d): # g is the games dataframe with predictions
                                             # d is the dictionary with run attributes to log
    # 1. Capture results spanning the entire dataset (all seasons)
    num_games = len(g)
    num_correct = g['Prediction Correct?'].values.sum()
    home_win_percent = g['Home Winner'].values.sum() / num_games * 100
    home_predict_percent = g['Home Wins?'].values.sum() / num_games * 100
    percent_correct = num_correct/num_games*100.
    
    new_row = pd.Series({'Predictor':'Net or Raw Points (see Net)',
                         'Season':0, # Indicates all seasons in the dataset.
                         'Lookback':ra_d['Lookback'],
                         'Net':ra_d['Net'],
                         'Num Games':num_games,
                         'Home Win %':home_win_percent,
                         'Home Predict %':home_predict_percent,
                         'Num Correct':num_correct,
                         '% Correct':percent_correct,
                         'Comment':('All Seasons' + ra_d['Comment'])})
    outcomes_df = outcomes_df.append(new_row, ignore_index=True)
    
    # 2. Caculate individual seasons represented in the dataset
    seasons = g['Date'].transform(lambda x: int(x/10000)).unique().tolist()
#     pprint.pprint(seasons)
    
    for season in seasons:
        start_date = int(season)*10000 + 101  # YYYYMMDD, Jan 01 of the Year
        end_date = int(season)*10000 + 1231   # YYYYMMDD, DEC 31 of the Year
        season_df = g.loc[((g['Date']>= start_date) &(g['Date']<= end_date)), :]
        
        num_games = len(season_df)
        num_correct = season_df['Prediction Correct?'].values.sum()
        home_win_percent = season_df['Home Winner'].values.sum() / num_games * 100
        home_predict_percent = season_df['Home Wins?'].values.sum() / num_games * 100
        percent_correct = num_correct/num_games*100.
        
        new_row = pd.Series({'Predictor':ra_d['Predictor'],
                     'Season':season,
                     'Lookback':ra_d['Lookback'],
                     'Net':ra_d['Net'],
                     'Num Games':num_games,
                     'Home Win %':home_win_percent,
                     'Home Predict %':home_predict_percent,
                     'Num Correct':num_correct,
                     '% Correct':percent_correct,
                     'Comment':('All Seasons' + ra_d['Comment'])})
        
        outcomes_df = outcomes_df.append(new_row, ignore_index=True)
        
    return outcomes_df

In [ ]:
# Predict games based on net points or raw points over a lookback window, visitor vs home.
def net_point_predictor(outcomes_df, lookback_n, prior_exclude=0, net=True):
    # Let's get started. Read the dataset.
    games_df = read_source_data()
    
    num_games = len(games_df)
    home_wins = games_df['Home Winner'].sum()
    home_wins_percent = home_wins / num_games * 100.
    print(f'Baseline results: {num_games} games, {home_wins} home wins = {round(home_wins_percent, 2)}%')
    
    games_df.index.set_names('games_idx', inplace=True)
    
    # Calculate rolling average of net points excluding the current row (shift=prior_exclude+1)
    if net:
        games_df = games_df.sort_values(by = ['Visiting Team','games_idx'])
        games_df['V NP Avg'] = games_df.groupby('Visiting Team')['V NetRuns'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())

        games_df = games_df.sort_values(by = ['Home Team','games_idx'])
        games_df['H NP Avg'] = games_df.groupby('Home Team')['H NetRuns'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
    else:
        games_df = games_df.sort_values(by = ['Visiting Team','games_idx'])
        games_df['V NP Avg'] = games_df.groupby('Visiting Team')['Visiting Score'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())

        games_df = games_df.sort_values(by = ['Home Team','games_idx'])
        games_df['H NP Avg'] = games_df.groupby('Home Team')['Home Score'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
       
    games_df = games_df.sort_index()

    print(f"  # of games before dropping lookback window: {len(games_df)}")
    games_df.to_csv("output/result_before_dropna.csv")
    games_df = games_df.dropna(how='any')
    print(f"  # of games after dropping lookback window: {len(games_df)}")

    games_df['Home Wins?'] = games_df['H NP Avg'] >= games_df['V NP Avg']
    games_df['Prediction Correct?'] = games_df['Home Wins?'] == games_df['Home Winner']
    
    # Setup run attributes so they can be included in the outcomes_df generated.
#     run_attributes = {}
#     run_attributes['Predictor'] = 'Net or Raw Points (see Net)'
#     run_attributes['Lookback'] = n
#     run_attributes['Net'] = net
#     run_attributes['Comment'] = ''
    run_attributes = {'Predictor':'Points', 'Lookback':n, 'Net':False, 'Comment':''}
    
    return append_outcomes(outcomes_df, games_df, run_attributes)


In [ ]:
# Predict games based on pitcher performance over a lookback window, visitor vs home.
def pitcher_predictor(outcomes_df, lookback_n):
    # Let's get started. Read the dataset.
    g = read_source_data()
    
    num_games = len(g)
    home_wins = g['Home Winner'].sum()
    home_wins_percent = home_wins / num_games * 100.
    print(f'Baseline results: {num_games} games, {home_wins} home wins = {round(home_wins_percent, 2)}%')
    
    g.index.set_names('games_idx', inplace=True)
    
    
    # Calculate rolling average of net points excluding the current row (shift=prior_exclude+1)
    g = g.sort_values(by = ['V Start Pitcher ID', 'games_idx'])
    g['V Runs Allowed'] = g.groupby('V Start Pitcher ID')['Home Score'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
#     print(g.head())

    g = g.sort_values(by = ['H Start Pitcher ID', 'games_idx'])
    g['H Runs Allowed'] = g.groupby('H Start Pitcher ID')['Visiting Score'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
#     print(g.head())
    
    g = g.sort_index()

    # Todo: For na's due to a lack of pitcher data, we should substitute data, not remove.
    print(f"  # of games before dropping lookback window: {len(g)}")
    num_na = g['V Runs Allowed'].isna().sum()
    print(f'num_na = {num_na}')
    
    
    g.to_csv("output/result_before_dropna.csv")
    # g = g.dropna(how='any')
    g = g.fillna(999)
    print(f"  # of games after dropping lookback window: {len(g)}")
    # print("  Warning, na's dropped that should be substituted instead.")

    g['Home Wins?'] = g['H Runs Allowed'] <= g['V Runs Allowed']
    g['Prediction Correct?'] = g['Home Wins?'] == g['Home Winner']
    
    # Setup run attributes so they can be included in the outcomes_df generated.
#     run_attributes = {}
#     run_attributes['Predictor'] = 'Pitcher'
#     run_attributes['Lookback'] = n
#     run_attributes['Net'] = False
#     run_attributes['Comment'] = ''
    run_attributes = {'Predictor':'Pitcher', 'Lookback':n, 'Net':False, 'Comment':''}
    
#     base_path = str(datetime.datetime.now()).replace(':', '-').replace(' ', '_')
#     data_output_path = "debug-pitcher-output/" + base_path + "_output.csv"
#     g.to_csv(path_or_buf=data_output_path)
    
    return append_outcomes(outcomes_df, g, run_attributes)


In [ ]:
# Top level (main) code, read in the dataset, add the columns we care about.
outcomes_df = pd.DataFrame(columns=['Season', 'Lookback', 'Net', 'Num Games',
                                  'Num Correct', '% Correct', 'Comment'])
# outcomes2_df = pd.DataFrame(columns=['Season', 'Lookback', 'Net', 'Num Games',
#                                   'Num Correct', '% Correct', 'Comment'])

# for n in [1, 2, 5, 10, 15, 20, 50, 60, 70, 80, 90, 100, 150, 200, 300, 400, 500]:
# for n in [50, 60, 70, 80, 90, 100]:
# for n in [5, 10, 25, 50, 75, 100, 150, 200]:
for n in [5, 10, 20, 40, 60, 80, 100]:
# for n in [5]:
    prior_exclude = 0

    if True:
        net=False
        outcomes_df = net_point_predictor(outcomes_df, n, prior_exclude, net=net)
        print(f"  Net Point Predictor, lookback={n}, prior exclude={prior_exclude}, net={net} done.")
        print("")
    else:
        outcomes_df = pitcher_predictor(outcomes_df, n)

outcomes_df


In [ ]:
seasons = outcomes_df['Season'].unique().tolist()
seasons

In [ ]:
season_gb = outcomes_df.groupby('Season')
type(season_gb)
y = season_gb.get_group(0)['% Correct']
print(y)
x = season_gb.get_group(0)['Lookback']
print(x)

In [ ]:
fig = plt.figure(figsize=(12,8))

axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])

axes.set_title('Prediction performance by lookback window length')
axes.set_xlabel('Lookback window size')
axes.set_ylabel('% Correct')

# Todo: Need to represent the predictor type in the plot text!

# for season in range(2012, 2018):
for season in outcomes_df['Season'].unique().tolist():
    x = season_gb.get_group(season)['Lookback']
    y = season_gb.get_group(season)['% Correct']
    axes.plot(x, y, label=str(season), marker='o')
    
axes.legend(loc='upper right')
axes.grid()
plt.show()

In [ ]:
if True:
    base_path = str(datetime.datetime.now()).replace(':', '-').replace(' ', '_')
    data_output_path = "output/" + base_path + "_output.csv"
    outcomes_df.to_csv(path_or_buf=data_output_path)
    plot_output_path = "plot/" + base_path + "_plot.png"
    fig.savefig(plot_output_path)

In [ ]:
# Regression Analysis
def net_points_regression(lookback_n, prior_exclude=0):
    # Let's get started. Read the dataset.
    games_df = read_source_data()
    
    # Caculate seasons represented in the dataset
    seasons = games_df['Date'].transform(lambda x: int(x/10000)).unique().tolist()
    pprint.pprint(seasons)
    

    # Calculate rolling average of net points excluding the current row (shift=prior_exclude+1)
    games_df = games_df.sort_values(by = ['Visiting Team','Date'])
    games_df['V NP Avg'] = games_df.groupby('Visiting Team')['V NetRuns'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
    games_df['V P Avg'] = games_df.groupby('Visiting Team')['Visiting Score'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())
    
    games_df = games_df.sort_values(by = ['Home Team','Date'])
    games_df['H NP Avg'] = games_df.groupby('Home Team')['H NetRuns'].transform(lambda x: x.shift(periods=prior_exclude+1).rolling(window = lookback_n).mean())

    games_df = games_df.sort_index()

    print(f"  # of games before dropping lookback window: {len(games_df)}")
    games_df.to_csv("output/result_before_dropna.csv")
    games_df = games_df.dropna(how='any')
    print(f"  # of games after dropping lookback window: {len(games_df)}")

    
    # The data table is built, now let's do the regression
    x = games_df['V NP Avg']
#     y = games_df['V NetRuns']
    y = games_df['V P Avg']
    slope, y_int, r, p, se = stats.linregress(x, y)
    
    print(f"slope: {slope}, int: {y_int}, r: {r}, r2: {r**2}, p: {p}, std error of the mean: {se}")
    
    
    
#     games_df['Pred V NP']
    
    
#     games_df['Home Wins?'] = games_df['H NP Avg'] >= games_df['V NP Avg']
#     games_df['Prediction Correct?'] = games_df['Home Wins?'] == games_df['Home Winner']

# This calls the above code
if False:
    net_points_regression(80)

In [ ]:
# From Venkat, on pitchers
# v_pitchers = game_day_df2['V Start Pitcher ID'].to_list()
# v_pitcher_net_score = []
# for pitcher in v_pitchers:
#     v_pitcher_net_score.append(train_df.loc[(train_df['V Start Pitcher ID']==pitcher),:]['Home Score'].mean())
# game_day_df2['V Avg Pitcher Runs given'] = v_pitcher_net_score
# h_pitchers = game_day_df2['H Start Pitcher ID'].to_list()
# h_pitcher_net_score = []
# for pitcher in h_pitchers:
#     h_pitcher_net_score.append(train_df.loc[(train_df['H Start Pitcher ID']==pitcher),:]['Visiting Score'].mean())
# game_day_df2['H Avg Pitcher Runs given'] = h_pitcher_net_score
# game_day_df2['Predict Home Pitcher Wins?'] = game_day_df2['H Avg Pitcher Runs given'] < game_day_df2['V Avg Pitcher Runs given']
# game_day_df2